In [55]:
import yasa
import mne
import matplotlib.pyplot as plt
import numpy as np
import statistics
import pybv
from scipy import signal
from scipy import io
import pymatreader
from mne.io import Raw
from mne import read_events
import wonambi
import h5py
import csv
import pandas as pd
from collections import Counter
import os

In [2]:
# some preliminaires
datapath = r'C:\Users\siann\Data\spindle_ppTMS\EEG'
subjects = 'sub-02'
session = ['ses-exp_01', 'ses-exp_02', 'ses-exp_03']
condition_peak = 'S155'
condition_trough = 'S156'
condition_rising = 'S157'
condition_falling = 'S158'
condition_sp_free = 'S159'
sf = 5000
os.chdir(datapath + '\\' + subjects)

We want to identify all spindle events that are nested within slow oscillations vs. spindle events occuring in isolation. First, we we use YASA to automatically detect slow oscillations in our data. Then, for each TMS pulse given, we identify whether the spindle targeted on that trial was accompanied by a slow oscillation or not.  

In [29]:
# read in the data cleaned of TMS artifacts as one continous dataset
raw_data = mne.io.read_raw_fif(datapath + '/' + subjects + '/' + 'data_tms_clean_unsegmented.fif', preload = True)
print(raw_data.info)
raw_data.filter(0.1, 40, picks=raw_data.ch_names) # bp filter
# and get trials based on stimulation times and conditions     
events = read_events(datapath + '/' + subjects + '/' + 'data_tms_clean_segmented.fif')
print(events)

# rereference?
# change semgentation of data to be -0.504 to -0.004s prior to TMS pulse

Opening raw data file C:\Users\siann\Data\spindle_ppTMS\EEG/sub-02/data_tms_clean_unsegmented.fif...
Isotrak not found


    Range : 0 ... 1844999 =      0.000 ...   369.000 secs
Ready.
Reading 0 ... 1844999  =      0.000 ...   369.000 secs...


C:\Users\siann\AppData\Local\Temp\ipykernel_8988\1134391566.py:2: RuntimeWarning: This filename (C:\Users\siann\Data\spindle_ppTMS\EEG/sub-02/data_tms_clean_unsegmented.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_data = mne.io.read_raw_fif(datapath + '/' + subjects + '/' + 'data_tms_clean_unsegmented.fif', preload = True)


<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F3, F4, C3, C4, P3, P4, O1, O2, F7, F8, T7, T8, P7, ...
 chs: 70 misc
 custom_ref_applied: False
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 lowpass: 2500.0 Hz
 meas_date: 1970-01-01 00:00:00 UTC
 meas_id: 4 items (dict)
 nchan: 70
 projs: []
 sfreq: 5000.0 Hz
>
No data channels found. The highpass and lowpass values in the measurement info will not be updated.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 165001 samples (33.000 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.1s


[[      0       0       2]
 [  22500       0       5]
 [  45000       0       4]
 [  67500       0       3]
 [  90000       0       1]
 [ 112500       0       2]
 [ 135000       0       1]
 [ 157500       0       4]
 [ 180000       0       5]
 [ 202500       0       3]
 [ 225000       0       3]
 [ 247500       0       5]
 [ 270000       0       2]
 [ 292500       0       1]
 [ 315000       0       4]
 [ 337500       0       5]
 [ 360000       0       3]
 [ 382500       0       2]
 [ 405000       0       1]
 [ 427500       0       4]
 [ 450000       0       2]
 [ 472500       0       3]
 [ 495000       0       1]
 [ 517500       0       5]
 [ 540000       0       4]
 [ 562500       0       3]
 [ 585000       0       2]
 [ 607500       0       4]
 [ 630000       0       1]
 [ 652500       0       5]
 [ 675000       0       4]
 [ 697500       0       1]
 [ 720000       0       2]
 [ 742500       0       5]
 [ 765000       0       1]
 [ 787500       0       4]
 [ 810000       0       5]
 

C:\Users\siann\AppData\Local\Temp\ipykernel_8988\1134391566.py:7: RuntimeWarning: This filename (C:\Users\siann\Data\spindle_ppTMS\EEG/sub-02/data_tms_clean_segmented.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = read_events(datapath + '/' + subjects + '/' + 'data_tms_clean_segmented.fif')


In [17]:
# yasa sleep stageing (not very accurate probably because data is artificially cut)
sleep_stages = yasa.SleepStaging(raw_data, eeg_name = 'C4', eog_name="VEOG", emg_name="EMG") 
values = sleep_stages.predict() # predict the sleep stages
from yasa import Hypnogram
hyp = Hypnogram(values, n_stages=5) # create yasa Hypnogram object


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\siann\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 0.24.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [28]:
# detect slow waves
hyp_up = hyp.upsample_to_data(sf = 1/30, data = raw_data)
data = raw_data.get_data() # get data from all channels
slow_oscillations = yasa.sw_detect(data, include=(1,2,3,4), hypno=hyp_up, sf = 5000, freq_sw=(0.3, 1.5),
                                      dur_neg=(0.3, 1.5), dur_pos=(0.1, 1), amp_neg=(40, 200), amp_pos=(10, 150),
                                      amp_ptp=(75, 350), coupling=False) # still change detection parameters?
slow_oscillations_df = slow_oscillations.summary()
print(slow_oscillations_df)


c:\Users\siann\AppData\Local\Programs\Python\Python39\lib\site-packages\yasa\hypno.py:483: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return self.hypno.replace(self.mapping).astype(np.int16)
c:\Users\siann\AppData\Local\Programs\Python\Python39\lib\site-packages\yasa\hypno.py:483: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  return self.hypno.replace(self.mapping).astype(np.int16)
04-Apr-24 15:50:57 | WARNING | Hypnogram is SHORTER than data by 9.00 seconds. Padding hypnogram with last value to match data.size.


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.6s
04-Apr-24 15:51:08 | WARNING | No SW were found in channel CHAN030.
04-Apr-24 15:51:08 | WARNING | No SW were found in channel CHAN031.


        Start   NegPeak  MidCrossing   PosPeak       End  Duration  \
0    315.7762  316.6014     317.1204  317.8008  318.0658    2.2896   
1    318.0658  318.2980     318.5648  319.2064  319.4512    1.3854   
2    319.4512  319.7290     320.6218  320.8006  321.0880    1.6368   
3    354.0116  354.3332     354.7268  355.2946  355.6060    1.5944   
4    355.6060  355.7954     356.0880  356.7318  356.9660    1.3600   
..        ...       ...          ...       ...       ...       ...   
200  340.3000  340.6172     341.0592  341.2300  341.3954    1.0954   
201  367.3694  367.6516     367.9054  368.1066  368.3434    0.9740   
202  304.9840  305.1900     305.4386  305.5430  305.6568    0.6728   
203  308.8194  309.1764     309.8338  310.0104  310.2268    1.4074   
204  349.3692  349.6500     349.9096  350.1280  350.8718    1.5026   

     ValNegPeak  ValPosPeak         PTP       Slope  Frequency  Stage  \
0    -70.579018  118.978570  189.557588  365.236200   0.436758      3   
1   -110.2787

In [67]:
# get stimulation times
stimulation_times_list = []
trial_start_list = []
condition_list = []
for i in events:
        stimulation_time = (i[0] + 2.5*sf) // sf  # get stimulation time in seconds 
        stimulation_times_list.append(stimulation_time)  
        trial_start = (i[0]) // sf  
        trial_start_list.append(trial_start)  
        condition = i[2]
        condition_list.append(condition)

# count how many times stimulation occured as the same time as detected SO and store in df
stimulation_times_df = pd.DataFrame({'Stimulation Times (sec)': stimulation_times_list, 'Trial start (sec)': trial_start_list, 
                                    'condition': condition_list})
stimulation_times_df['so_occuring'] = False
stimulation_times_df['so_occuring_trial_index'] = np.NaN
stimulation_times_df['so_occuring_trough_time'] = np.NaN
for index, entry in slow_oscillations_df.iterrows():
    start_time = entry['Start']
    end_time = entry['End']
    for i, row  in stimulation_times_df.iterrows():
        if (start_time <= row['Stimulation Times (sec)'] <= end_time): 
            stimulation_times_df.at[i, 'so_occuring'] = True
            stimulation_times_df.at[i, 'so_occuring_trial_index'] = index
            stimulation_times_df.at[i, 'so_occuring_trough_time'] = entry['NegPeak']
            stimulation_times_df.at[i, 'so_occuring_trough_time'] = entry['NegPeak']
            stimulation_times_df.at[i, 'so_occuring_trough_time'] = entry['NegPeak']
number_co_occurences = Counter(stimulation_times_df['so_occuring'])
print(number_co_occurences)
print(stimulation_times_df)
# write to csv
# stimulation_times_df.to_csv(path_or_buf=(datapath + '\\' + subjects ), sep=',', na_rep='', float_format=None, columns=None, header=True, 
                 # index=True, index_label=None, mode = 'w')
stimulation_times_df.to_csv('stimulation_times_so_cooccur.csv')

Counter({False: 76, True: 6})
    Stimulation Times (sec)  Trial start (sec)  condition  so_occuring  \
0                       2.0                  0          2        False   
1                       7.0                  4          5        False   
2                      11.0                  9          4        False   
3                      16.0                 13          3        False   
4                      20.0                 18          1        False   
..                      ...                ...        ...          ...   
77                    349.0                346          4        False   
78                    353.0                351          2        False   
79                    358.0                355          5         True   
80                    362.0                360          3         True   
81                    367.0                364          1        False   

    so_occuring_trial_index  so_occuring_trough_time  
0                       Na